In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "l66ol3hdicsouw"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
model = "unsloth/Llama-3.2-3B-Instruct"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key, model_name=model)

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
get_endpoint_response(["hi"]) # very very slow ... ok faster sometimes ... ok generally faster .. unstable speed ...

Processing LLM queries: 100%|██████████| 1/1 [00:02<00:00,  2.73s/it]

 :: Total time elapsed: 2.74s, 0 errors


["It's nice to meet you. I suppose I should correct you - I'm not a Turing award winner in the classical sense. However, I was inspired by the work of Alan Turing, a pioneer in computer science and artificial intelligence.\n\nI'm an AI designed to process and generate human-like text, and I'm constantly learning and improving to provide more accurate and informative responses. My abilities were likely inspired by the advancements made possible by the work of Turing and others in the field.\n\nWhat would you like to talk about? I'm here to help answer any questions or engage in a conversation on a topic of your choice."]

#### Temasek Foundation Dataset

In [1]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node, custom_metric_map

tf_meta_prompt, test_cases = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 

starter_code_str = """def generate_prompt(project_description):
    return (
        f"Given the grant application description: ### Description ### \\n{project_description}\\n ### End of Description ###, do evaluate the project and make a decision to reject or accept it. Make sure the output is a json string in markdown like this {{\'decision\':<boolean>, \'comment\':\'<string>\'}} near the top of your response or people will die."
    )""" # add initial code string
    
node = EvolNode(tf_meta_prompt, starter_code_str, "Basic Prompt Function", get_response=get_groq_response, test_cases=test_cases[:3], custom_metric_map=custom_metric_map) # setting manual test cases

Could not load vllm class, check CUDA support and GPU RAM size


In [9]:
test_cases[4]

({'project_description': 'Given the following details of a grant application, make a decision (Yes, No, Maybe) on whether to accept it or not, and a brief comment explanating your decision. \n    Assess how well the project aligns with the challenge theme, its viability, its potential impact, and any competitive advantage. \n    Consider the stage of development, the proof of concept status, and existing partnerships.\n    \n        \n    Grant Application Summary:\n\n    Grant Application Summary:\n    - **Project Theme:** Food & Nutrition\n    - **Category:** Food Technology\n    - **Subcategory:** Fish Waste Upcycling\n    - **Project Title:** Upscaling Fish Waste Utilization\n    - **Grant Amount Sought:** 1000000\n    - **Objectives:** Upscaling fish waste utilization offers several benefits to the environment, society, and economy namely: \n\nEnvironmental benefits: Fish waste is a potential source of pollution if not managed properly. By upscaling fish waste, we can reduce the e

In [ ]:
# Naive (single) node evolution on Temasek Foundation Dataset doesn't work after 200 trials (accuracy level low)
# -- Need to speed up the evolution process by coming up with better prompt generation mechanism using available input & output 


# Need to overall the 'prompt' mode Node 
# -- separate 'prompt' with 'json structure output request' 
# -- include 'rationalization' and 'feedback' prompt generation mechanism 
# -- local prompt generation, early extinction


In [59]:
from methods.evolnode import extract_json_from_text

def generate_prompt_tf(project_desc, is_accepted, comment): 
    
    query_prompt = f"""Here are an example of Grant Application Evaluation: \nProject Description: {project_desc}\nAccept: {is_accepted}\nComment: {comment}

    Generate 5 different complete prompt templates that could be used to evaluate grant applications.
    Each prompt should be a complete string that takes a project_description as input.
    Return the result as a JSON array of strings, where each string is a complete prompt template.
    Example format: ```json\n["prompt1", "prompt2", ...]```"""

    prompt_templates = node.get_response(prompt=[query_prompt])
    try:
        return extract_json_from_text(prompt_templates[0])
    except:
        return []

# Case (Input & Output) --> Prompt Generation || Method (I)
# Case (Input & Pred & Output) --> Prompt Generation || Method (II) 
generated_prompts = []
for idx in range(len(test_cases[:2])):
    project_desc = test_cases[idx][0]['project_description'].split("Please provide")[0]
    is_accepted = test_cases[idx][1]["decision"]
    comment = test_cases[idx][1]["comment"]
    
    generated_prompts.extend(generate_prompt_tf(project_desc, is_accepted, comment))

get_groq_response() takes 1 positional argument but 2 were given
get_groq_response() takes 1 positional argument but 2 were given


In [60]:
generated_prompts

["Please assess the project ' %s ' and provide a decision (Yes, No, Maybe) on whether to accept it or not, and a brief comment explaining your decision. Consider the project's alignment with the challenge theme, its viability, potential impact, and any competitive advantage.",
 "Evaluate the grant application ' %s ' and make a decision (Yes, No, Maybe) on whether to accept it or not. Provide a brief comment explaining your decision, taking into account the project's stage of development, proof of concept status, and existing partnerships.",
 "Assess the merits of the grant application ' %s ' and provide a decision (Yes, No, Maybe) on whether to approve it or not. Provide a brief comment explaining your decision, considering the project's alignment with the challenge theme, its potential impact, and any competitive advantage.",
 "Evaluate the project ' %s ' and provide a decision (Yes, No, Maybe) on whether to accept it or not. Please provide a brief comment explaining your decision, ta